In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100%;}</style>"))

In [52]:
import pandas as pd
import inspect 
pd.set_option('display.max_columns',None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
u_cols=['user_id','age','sex','occupation','zipcode']
users=pd.read_csv('ml-100k/u.user',sep='|',names=u_cols,encoding='latin-1')

In [19]:
users.head()

,user_id,age,sex,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [22]:
r_cols=['user_id','movie_id','rating','unix_timestamp']
ratings=pd.read_csv('ml-100k/u.data',names=r_cols,sep='\t')
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [36]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items=pd.read_csv('ml-100k/u.item',sep='|',names=i_cols,encoding='latin-1')
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [37]:
users.shape,items.shape,ratings.shape

((943, 5), (1682, 24), (100000, 4))

In [38]:
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [44]:
n_users=ratings.user_id.nunique()
n_items=ratings.movie_id.nunique()
n_users,n_items

(943, 1682)

In [48]:
ratings.head(2)

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [49]:
data_matrics=np.zeros((n_users,n_items))
# data_matrics[:10]
for line in ratings.itertuples():
    data_matrics[line[1]-1,line[2]-1]=line[3]

In [50]:
data_matrics[:5]

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 3., 0., ..., 0., 0., 0.]])

In [102]:
from numpy.linalg import norm
def cosine_similarity(shape,data_matrics):
    check=np.zeros((shape,shape))
    for j in range(shape):
        vec1=data_matrics[j]
        for i in range(shape):
            vec2=data_matrics[i]
            check[j,i]=(1-np.dot(vec1,vec2)/(norm(vec1)*norm(vec2)))
    return check

In [93]:
# np.dot(vec1,vec2)/(norm(vec1)*norm(vec2))
user_similarity=cosine_similarity(n_users,data_matrics)

In [100]:
item_similarity=cosine_similarity(n_items,data_matrics.T)

In [103]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'unix_timestamp'], dtype='object')

In [105]:
# ratings.mean(axis=1)

In [129]:
def prediction(ratings,similarity,types):
    if(types=='user'):
        means_user_rating=ratings.mean(axis=1)
        rating_diff=ratings-means_user_rating[:,np.newaxis]
        pred=means_user_rating[:,np.newaxis]+similarity.dot(rating_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
        return pred
    elif types=='item':
        return ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])

In [130]:
user_prediction=prediction(data_matrics,user_similarity,'user')

In [131]:
item_prediction=prediction(data_matrics,item_similarity,'item')